In [65]:
import ciao_contrib.runtool as rt
import sherpa.astro.ui as ui
import numpy as np
import matplotlib.pyplot as plt
import os

#astropy is used to read WCS position information from fits files for region_count
from astropy.io import fits
from astropy import wcs
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance


def reproject(obsid_in, obsid_ref):

    reproj = rt.reproject_aspect
    reproj.punlearn()
    reproj.infile = f'data/{obsid_in}/GW170817/GW170817_src_6.fits'
    reproj.refsrcfile = f'data/{obsid_ref}/GW170817/GW170817_src_6.fits'
    reproj.updfile = f'data/{obsid_in}/repro/acisf{obsid_in}_repro_evt2.fits'
    reproj.wcsfile = f'data/{obsid_in}/repro/acisf{obsid_in}_repro_evt2.fits'
    reproj.residlim = 0.5 #Half ArcSec limit
    reproj.outfile = f'data/{obsid_in}/GW170817/GW170817_reproj.fits'
    reproj()



def region_count(obsid_in):

    srcflx = rt.srcflux
    srcflx.punlearn() #Restore the system defaults for a CIAO parameter file.
    srcflx.infile = f'data/{obsid_in}/repro/acisf{obsid_in}_repro_evt2.fits'
    with fits.open(srcflx.infile) as f:
        loc = SkyCoord(f[1].header['RA_NOM']*u.deg,f[1].header['DEC_NOM']*u.deg)
        loc_str = loc.ra.to_string(unit=u.hourangle, sep=':', pad=True, precision=2) + " " + loc.dec.to_string(unit=u.degree, sep=':', pad=True, precision=2)
    srcflx.pos = loc_str #Reads RA, DEC columns as a position in sexagesimal "degree:arcmin:arcsec" format
    srcflx.outroot = f'data/{obsid_in}/repro/'
    srcflx.srcreg = f'data/20860/repro/srcwcs.reg' #region used to calculate the source counts
    srcflx.bkgreg = f'data/20860/repro/bkgwcs.reg'#region used to calculate the background counts
    srcflx.clobber = 'yes'
    srcflx.verbose = 0 #Default =1, 0 suppress the long print statement
    return(srcflx())


In [66]:
#reproject(20936, 20860)

region_count(20861)